# Google Firestore（Datastore 模式）

> [Firestore（Datastore 模式）](https://cloud.google.com/datastore) 是一款为自动扩展、高性能和简易应用开发而构建的 NoSQL 文档数据库。利用 Datastore 的 Langchain 集成，扩展您的数据库应用以构建支持 AI 的体验。

本 Notebook 将介绍如何使用 [Firestore（Datastore 模式）](https://cloud.google.com/datastore) 通过 `DatastoreLoader` 和 `DatastoreSaver` 来[保存、加载和删除 Langchain 文档](/docs/how_to#document-loaders)。

在 [GitHub](https://github.com/googleapis/langchain-google-datastore-python/) 上了解更多关于该软件包的信息。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-datastore-python/blob/main/docs/document_loader.ipynb)

## 开始之前

要运行此笔记本，您需要完成以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Datastore API](https://console.cloud.google.com/flows/enableapi?apiid=datastore.googleapis.com)
* [创建 Datastore 模式的 Firestore 数据库](https://cloud.google.com/datastore/docs/manage-databases)

在确认笔记本的运行时环境可以访问数据库后，请填充以下值并运行单元格，然后再运行示例脚本。

### 🦜🔗 库安装

集成位于我们自己的 `langchain-google-datastore` 包中，因此我们需要安装它。

In [ ]:
%pip install -upgrade --quiet langchain-google-datastore

**仅限 Colab**: 取消注释以下单元格以重新启动内核，或使用按钮重新启动内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重新启动终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此 notebook 中利用 Google Cloud 资源。

如果您不知道项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 身份验证

以登录此笔记本的 IAM 用户身份验证到 Google Cloud，以便访问您的 Google Cloud 项目。

- 如果您正在使用 Colab 运行此 Notebook，请使用下面的单元格并继续。
- 如果您正在使用 Vertex AI Workbench，请查看此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

## 基本用法

### 保存文档

使用 `DatastoreSaver.upsert_documents(<documents>)` 保存 Langchain 文档。默认情况下，它会尝试从 `Document` 元数据中的 `key` 中提取实体键。

In [ ]:
from langchain_core.documents import Document
from langchain_google_datastore import DatastoreSaver

saver = DatastoreSaver()

data = [Document(page_content="Hello, World!")]
saver.upsert_documents(data)

#### 无密钥保存文档

如果指定了 `kind`，文档将使用自动生成的 ID 进行存储。

In [ ]:
saver = DatastoreSaver("MyKind")

saver.upsert_documents(data)

### 通过 Kind 加载文档

使用 `DatastoreLoader.load()` 或 `DatastoreLoader.lazy_load()` 加载 langchain 文档。`lazy_load` 返回一个生成器，该生成器仅在迭代期间查询数据库。要初始化 `DatastoreLoader` 类，您需要提供：
1. `source` - 加载文档的源。它可以是 Query 类的实例，也可以是要从中读取的 Datastore 的 kind 名称。

In [6]:
from langchain_google_datastore import DatastoreLoader

loader = DatastoreLoader("MyKind")
data = loader.load()

### 通过查询加载文档

除了从 kind 加载文档外，我们还可以选择从查询加载文档。例如：

In [ ]:
from google.cloud import datastore

client = datastore.Client(database="non-default-db", namespace="custom_namespace")
query_load = client.query(kind="MyKind")
query_load.add_filter("region", "=", "west_coast")

loader_document = DatastoreLoader(query_load)

data = loader_document.load()

### 删除文档

使用 `DatastoreSaver.delete_documents(<documents>)` 从 Datastore 删除 langchain 文档列表。

In [ ]:
saver = DatastoreSaver()

saver.delete_documents(data)

keys_to_delete = [
    ["Kind1", "identifier"],
    ["Kind2", 123],
    ["Kind3", "identifier", "NestedKind", 456],
]
# The Documents will be ignored and only the document ids will be used.
saver.delete_documents(data, keys_to_delete)

## 高级用法

### 使用自定义的文档页面内容和元数据加载文档

`page_content_properties` 和 `metadata_properties` 的参数将指定要写入 LangChain Document 的 `page_content` 和 `metadata` 的实体属性。

In [8]:
loader = DatastoreLoader(
    source="MyKind",
    page_content_fields=["data_field"],
    metadata_fields=["metadata_field"],
)

data = loader.load()

### 自定义页面内容格式

当 `page_content` 只包含一个字段时，信息将仅显示该字段的值。否则，`page_content` 将采用 JSON 格式。

### 自定义连接与认证

In [ ]:
from google.auth import compute_engine
from google.cloud.firestore import Client

client = Client(database="non-default-db", creds=compute_engine.Credentials())
loader = DatastoreLoader(
    source="foo",
    client=client,
)